# Chapitre 3 : Test sur les modèles
---

## 1- Imports et récupération des données

In [1]:
# Default
import pandas as pd
import seaborn as sns
import numpy as np
import warnings
import plotly.express as px
import joblib
import time
import os

# machine learning - scikit learn:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, RobustScaler, MinMaxScaler
from sklearn import set_config
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

from sklearn.neighbors import KNeighborsClassifier
from sklearn.experimental import enable_hist_gradient_boosting  # activer HistGradientBoostingClassifier


# désactiver temporairement les avertissements
warnings.filterwarnings('ignore')

/home/mikaleff/Bureau/Airline_Jeremy/env_sentiment_analysis/lib/python3.11/site-packages/sklearn/experimental/enable_hist_gradient_boosting.py:16: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


In [2]:
# Récupération des données sur https://raw.githubusercontent.com/remijul/dataset/master/Airline%20Passenger%20Satisfaction.csv
file_path = "https://raw.githubusercontent.com/remijul/dataset/master/Airline%20Passenger%20Satisfaction.csv"
df = pd.read_csv(file_path, sep=";", index_col=False)
df = df.drop('id', axis=1)

### Dataframe de base

In [3]:
df

,Satisfaction,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Seat comfort,Departure/Arrival time convenient,Food and drink,...,Online support,Ease of Online booking,On-board service,Leg room service,Baggage handling,Checkin service,Cleanliness,Online boarding,Departure Delay in Minutes,Arrival Delay in Minutes
0,satisfied,Female,Loyal Customer,65,Personal Travel,Eco,265,0,0,0,...,2,3,3,0,3,5,3,2,0,0.0
1,satisfied,Male,Loyal Customer,47,Personal Travel,Business,2464,0,0,0,...,2,3,4,4,4,2,3,2,310,305.0
2,satisfied,Female,Loyal Customer,15,Personal Travel,Eco,2138,0,0,0,...,2,2,3,3,4,4,4,2,0,0.0
3,satisfied,Female,Loyal Customer,60,Personal Travel,Eco,623,0,0,0,...,3,1,1,0,1,4,1,3,0,0.0
4,satisfied,Female,Loyal Customer,70,Personal Travel,Eco,354,0,0,0,...,4,2,2,0,2,4,2,5,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129875,satisfied,Female,disloyal Customer,29,Personal Travel,Eco,1731,5,5,5,...,2,2,3,3,4,4,4,2,0,0.0
129876,neutral or dissatisfied,Male,disloyal Customer,63,Personal Travel,Business,2087,2,3,2,...,1,3,2,3,3,1,2,1,174,172.0
129877,neutral or dissatisfied,Male,disloyal Customer,69,Personal Travel,Eco,2320,3,0,3,...,2,4,4,3,4,2,3,2,155,163.0
129878,neutral or dissatisfied,Male,disloyal Customer,66,Personal Travel,Eco,2450,3,2,3,...,2,3,3,2,3,2,1,2,193,205.0


### Changement des noms de colonne

In [4]:
# Changement du nom des colonnes
df.columns = ['Satisfaction', 'Gender', 'Customer_Type', 'Age', 'Type_of_Travel',
                    'Class', 'Flight_Distance', 'Seat_comfort', 'Departure_Arrival_time_convenient', 
                    'Food_and_drink', 'Gate_location', 'Inflight_wifi_service', 'Inflight_entertainment', 'Online_support', 
                    'Ease_of_Online_booking', 'On_board_service', 'Leg_room_service', 'Baggage_handling',
                    'Checkin_service', 'Cleanliness', 'Online_boarding', 'Departure_Delay_in_Minutes', 'Arrival_Delay_in_Minutes']

---
## 2- Modélisation

In [5]:
# Préparation du DataFrame pour modélisation et affichage des colonnes numériques et catégorielles
df_pour_modelisation = df.copy()
df_pour_modelisation = df_pour_modelisation.dropna() # Suppréssion des données manquantes

# Affichage des listes de features catégorielles et de features numériques
colonnes_num = list(df_pour_modelisation.select_dtypes(include=['int', 'float']).columns)
colonnes_cat = list(df_pour_modelisation.select_dtypes(include=['object']).columns)

print(f'Colonnes numériques : {colonnes_num}')
print(f'Colonnes catégorielles : {colonnes_cat}')

Colonnes numériques : ['Age', 'Flight_Distance', 'Seat_comfort', 'Departure_Arrival_time_convenient', 'Food_and_drink', 'Gate_location', 'Inflight_wifi_service', 'Inflight_entertainment', 'Online_support', 'Ease_of_Online_booking', 'On_board_service', 'Leg_room_service', 'Baggage_handling', 'Checkin_service', 'Cleanliness', 'Online_boarding', 'Departure_Delay_in_Minutes', 'Arrival_Delay_in_Minutes']
Colonnes catégorielles : ['Satisfaction', 'Gender', 'Customer_Type', 'Type_of_Travel', 'Class']


### Définition de notre X(données) et y(target)

In [6]:
# Définition de X et y
y = df_pour_modelisation['Satisfaction']
X = df_pour_modelisation.drop(['Satisfaction'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10) # taille du jeu test 30% et seed numéro 10

print("La longueur du dataset de base :", len(X))
print("La longueur du dataset d'entraînement :", len(X_train))
print("La longueur du dataset de test :", len(X_test))

La longueur du dataset de base : 129487
La longueur du dataset d'entraînement : 90640
La longueur du dataset de test : 38847


### Pré-processing

In [7]:
# Préparation des données
preparation = ColumnTransformer(
    transformers=[       
        ('data_cat',
         OneHotEncoder(handle_unknown='ignore'), ['Gender', 'Customer_Type', 'Type_of_Travel', 'Class']),
        ('data_num',
         StandardScaler(),colonnes_num),         
    ])

### Test de plusieurs modèles

In [8]:
# Définition des paramètres à ajuster et la Durée d'éxecution
models = {
    'LogisticRegression': LogisticRegression(),                                 # ~6s
    #'SVC': SVC(),                                                              # ~1149s trop long
    #'HistGradientBoostingClassifier': HistGradientBoostingClassifier(),         
    'KNeighborsClassifier': KNeighborsClassifier(),
    'RandomForestClassifier': RandomForestClassifier()                          # ~105s
}

params = {
    'LogisticRegression': {'classifier__C': [0.1, 1, 10]},
    #'SVC': {},
    #'HistGradientBoostingClassifier': {'classifier__max_depth': [3, 8], 'classifier__min_samples_leaf': [1, 10]},
    'KNeighborsClassifier': {},
    'RandomForestClassifier': {'classifier__max_depth': [10, 50, 100]}
}

In [10]:
# Efface l'ancien modèle enregistrer
if os.path.isfile('best_model.joblib'):  
     os.remove('best_model.joblib')

# Donne un aperçu des résultats des différents modèles
p=0
for model_name, model in models.items():
     p=p+1
     print(f"\n-----------------------------Modèle {p}---------------------------------\n")
     print(f"GridSearchCV for {model_name}")

     start_time = time.time()

     # Création du pipeline
     #model = RandomForestClassifier()
     model_lm = Pipeline([('preparation', preparation),
                         ('classifier',model)])

     # Application du gridsearchcv
     gs = GridSearchCV(model_lm, params[model_name], cv=5) 
     gs.fit(X_train, y_train)

     end_time = time.time()
     training_time = end_time - start_time

     print(f"Best params: {gs.best_params_}")                    # Renvoi les meilleurs paramètres
     print(f"Train score: {gs.best_score_*100:.3f}%")            # Renvoi le meilleur score d'accuracy sur les données d'entraînement
     print(f"Test score: {gs.score(X_test, y_test)*100:.3f}%\n") # Renvoi le meilleur score d'accuracy sur les données test

     cv_results = gs.cv_results_
     cv_results_df = pd.DataFrame(cv_results) 
     
     # Mise en page pour les dataframe de metrics d'erreur
     with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.float_format', '{:.3f}'.format):
          display(cv_results_df)

     print(f"\n------------------------------------------------------- {training_time:.2f} seconds")

# Récupération du meilleur modèle
best_model = gs.best_estimator_
joblib.dump(best_model, 'best_model.joblib')   



-----------------------------Modèle 1---------------------------------

GridSearchCV for LogisticRegression
Best params: {'classifier__C': 1}
Train score: 83.461%
Test score: 83.801%



,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_classifier__C,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.796,0.092,0.069,0.003,0.100,{'classifier__C': 0.1},0.839,0.832,0.836,0.834,0.832,0.835,0.003,3
1,0.616,0.029,0.081,0.012,1,{'classifier__C': 1},0.839,0.832,0.836,0.834,0.833,0.835,0.002,1
2,0.596,0.043,0.073,0.007,10,{'classifier__C': 10},0.839,0.832,0.836,0.834,0.833,0.835,0.002,2



------------------------------------------------------- 12.11 seconds

-----------------------------Modèle 2---------------------------------

GridSearchCV for KNeighborsClassifier
Best params: {}
Train score: 92.143%
Test score: 92.416%



,mean_fit_time,std_fit_time,mean_score_time,std_score_time,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.183,0.014,1.601,0.046,{},0.923,0.919,0.921,0.925,0.920,0.921,0.002,1



------------------------------------------------------- 9.27 seconds

-----------------------------Modèle 3---------------------------------

GridSearchCV for RandomForestClassifier
Best params: {'classifier__max_depth': 100}
Train score: 95.637%
Test score: 95.830%



,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_classifier__max_depth,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,5.626,0.192,0.218,0.015,10,{'classifier__max_depth': 10},0.935,0.935,0.935,0.933,0.930,0.934,0.002,3
1,7.185,0.456,0.316,0.031,50,{'classifier__max_depth': 50},0.959,0.956,0.957,0.955,0.954,0.956,0.001,2
2,7.263,0.559,0.313,0.039,100,{'classifier__max_depth': 100},0.957,0.957,0.956,0.957,0.955,0.956,0.001,1



------------------------------------------------------- 113.86 seconds


['best_model.joblib']

---
**Infos pour exploiter le cv_results_ de gridsearchcv**
- mean_test_score : moyenne des scores de validation croisée pour chaque combinaison de paramètres testée.
- std_test_score : écart type des scores de validation croisée pour chaque combinaison de paramètres testée.
- rank_test_score : le classement de chaque combinaison de paramètres testée en fonction de leur performance.
---

### Recherche du feature importance

In [ ]:
# Ne fonctionne qu'avec le RandomForestClassifier
feature_importances = gs.best_estimator_.named_steps['classifier'].feature_importances_
feature_names = gs.best_estimator_.named_steps['preparation'].get_feature_names_out()

feature_importance = pd.Series({feature_names[i] : feature_importances[i] for i in range(len(feature_importances))})

feature_importance.sort_values(ascending=False)
print(feature_importance)

---
## Conclusion
On choisis le meilleure modèle pour essayé de prédire si suivant les différents parametres, la personne qui répond au questionnaire va être satisfaite ou non. Dans notre cas j'utilise un gridsearchcv qui nous donne le meilleur score de validation croisé pour chacun des modèles avec chacune des combinaisons d'hyperparametre que je donne en amont.

les 5 modèles :
- **LogisticRegression :** fonctionne correctement et rapidement avec un score d'accuracy sur des données propres un peu moyen


- **RandomForestClassifier :**meilleur solution tant par la rapidité d'éxecution que la précision
- **SVC :** j'ai été obligé d'arrêter l'executions à plusieurs reprises car il prend beaucoup trop de temps
- **HistGradientBoostingClassifier :** je n'ai pas compris les résultats, il avais tendance a prendre le pas sur les autres malgré un score d'accuracy plus faible et au moment de la prédiction il n'afficher que non satisfait
- **KNeighborsClassifier :** n'est juste pas le meilleur modèle d'après mes sources
---

## Test shapash
*Problèmes de compatibilité avec python 3.11 => repassé en 3.10 mais trop tard dans le temps imparti du tp*

In [ ]:
from shapash import SmartExplainer

xpl = SmartExplainer(
  model=best_model,
  preprocessing=preparation, # Optional: compile step can use inverse_transform method
)

xpl.compile(
  x=X_test, 
  y_pred=y_test, # Optional: for your own prediction (by default: model.predict)
  y_target=y, # Optional: allows to display True Values vs Predicted Values
  additional_data=df, # Optional: additional dataset of features for Webapp
)

app = xpl.run_app()

xpl.generate_report(
    output_file='path/to/output/report.html',
    project_info_file='path/to/project_info.yml',
    x_train=X_train,
    y_train=y_train,
    y_test=y_test,
    title_story="House prices report",
    title_description="""This document is a data science report of the kaggle house prices tutorial project.
        It was generated using the Shapash library.""",
    metrics=[{‘name’: ‘MSE’, ‘path’: ‘sklearn.metrics.mean_squared_error’}]
)